In [1]:
!pip install pyspark;


In [2]:
import pyspark

# Know your machine resources

In [1]:
# CPU Cores
!sysctl -n hw.logicalcpu

# Memory in Bytes
!sysctl -n hw.memsize

8
17179869184


In [3]:
# Memory in GB
17179869184/(1024**3)

16.0

In [4]:
MAX_MEMORY = '4g'
MAX_MEMORY_OVERHEAD = '1g'
MAX_DRIVER_MEMORY = '1g'

In [ ]:
conf = pyspark.SparkConf().setMaster("local[2]") \
        # .set('spark.executor.heartbeatInterval', 10000) \
        # .set('spark.network.timeout', 10000) \
        # .set('spark.core.connection.ack.wait.timeout', '3600') \
        .set('spark.executor.memory', MAX_MEMORY) \
        .set('spark.executor.memoryOverhead', MAX_MEMORY_OVERHEAD) \
        .set('spark.driver.memory', MAX_DRIVER_MEMORY)